In [1]:
import numpy as np
import random
import patsy
import statsmodels.api as sm
from scipy.stats import chi2

## Exercise 2

In [2]:
def f(x1, x2):
    assert x1.size == x2.size
    return np.sin(x1) + np.cos(x2)

In [3]:
n2 = 1000
X = np.random.uniform(size=(n2, 2))
y = f(X[:, 0], X[:, 1]) + np.random.normal(scale=0.1, size=(n2, )) # shape (1000, )

In [4]:
def bx_tensor(x, m=2):
    knots = [(i/(m+1)) for i in range(1, m+1)]
    basis_num = m + 1 + 3
    bx1 = patsy.bs(x=x[:, 0], knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)
    bx2 = patsy.bs(x=x[:, 1], knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)
    tmp_list = [bx1[:,i].reshape((-1,1))*bx2 for i in range(basis_num)]
    output = np.hstack(tmp_list)
    return output

def get_fhat(x, y, m=2):
    bx = bx_tensor(x=x, m=m)
    return sm.OLS(y, bx).fit().predict()

In [5]:
bx_tensor(X).shape

(1000, 36)

In [6]:
bx_tensor_prediction = get_fhat(X, y)
bx_tensor_prediction.shape

(1000,)

In [7]:
knots = [(i/(2+1)) for i in range(1, 2+1)]
bsx = patsy.bs(x=X[:, 0], knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)
bsz = patsy.bs(x=X[:, 1], knots=knots, degree=3, include_intercept=False, lower_bound=0, upper_bound=1)
print(bsx.shape, bsz.shape)
design_matrix = np.hstack((bsx, bsz))
print(design_matrix.shape)

(1000, 6) (1000, 5)
(1000, 11)


In [8]:
lm_model = sm.OLS(y, design_matrix)
lm_results = lm_model.fit()

In [9]:
W = (lm_results.resid**2).sum() / ((y - bx_tensor_prediction)**2).sum()
print("n*log(W) =", (n2*np.log(W)).item())

n*log(W) = 23.823367880572007


In [10]:
print("p-values: ", 1-chi2.cdf(x=n2*np.log(W), df=25))

p-values:  0.5295975633221048


#### (c)

In [11]:
pvalue_listc = []
for i in range(500):
    np.random.seed(i)
    x = np.random.uniform(size=(1000, 2))
    ys = f(x[:, 0], x[:, 1]) + np.random.normal(scale=0.1, size=(n2, ))
    knots = [(i/(2+1)) for i in range(1, 2+1)]
    bsx = patsy.bs(x=x[:, 0], knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)
    bsz = patsy.bs(x=x[:, 1], knots=knots, degree=3, include_intercept=False, lower_bound=0, upper_bound=1)
    design_matrix = np.hstack((bsx, bsz))
    bx_tensor_prediction_simu = get_fhat(x, ys)
    lm_results_simu = sm.OLS(ys, design_matrix).fit()
    Ws = (lm_results_simu.resid**2).sum() / ((ys - bx_tensor_prediction_simu)**2).sum()
    pvalue_listc.append(1-chi2.cdf(x=n2*np.log(Ws), df=25))
pvalue_listc = np.array(pvalue_listc)

In [12]:
print("The probability of rejecting H0: ", np.where(pvalue_listc<0.05)[0].size/500)
print(f"500次模擬結果中，共{np.where(pvalue_listc<0.05)[0].size}次拒絕H0 (pvalue < 0.05)")

The probability of rejecting H0:  0.058
500次模擬結果中，共29次拒絕H0 (pvalue < 0.05)


#### (d)

In [13]:
def f2(x1, x2):
    return np.sin(x1)*np.cos(x2)

In [14]:
pvalue_listd = []
for i in range(500):
    np.random.seed(i)
    X_d = np.random.uniform(size=(n2, 2))
    knots = [(i/(2+1)) for i in range(1, 2+1)]
    bsx_d = patsy.bs(x=X_d[:, 0], knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)
    bsz_d = patsy.bs(x=X_d[:, 1], knots=knots, degree=3, include_intercept=False, lower_bound=0, upper_bound=1)
    design_matrix = np.hstack((bsx_d, bsz_d))
    yd = f2(X_d[:, 0], X_d[:, 1]) + np.random.normal(scale=0.1, size=(n2, ))
    bx_tensor_prediction_simu2 = get_fhat(X_d, yd)
    lm_results_simu2 = sm.OLS(yd, design_matrix).fit()
    Wd = (lm_results_simu2.resid**2).sum() / ((yd - bx_tensor_prediction_simu2)**2).sum()
    pvalue_listd.append(1-chi2.cdf(x=n2*np.log(Wd), df=25))
pvalue_listd = np.array(pvalue_listd)

In [15]:
print("The probability of rejecting H0: ", np.where(pvalue_listd<0.05)[0].size/500)
print(f"500次模擬結果中，共{np.where(pvalue_listd<0.05)[0].size}次拒絕H0 (pvalue < 0.05)")

The probability of rejecting H0:  1.0
500次模擬結果中，共500次拒絕H0 (pvalue < 0.05)
